In [1]:
import pandas as pd
import re
import spacy
from collections import Counter
import unicodedata
from bs4 import BeautifulSoup
from boilerpipe.extract import Extractor
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from nltk.corpus import stopwords
stopwords = set(stopwords.words())
ANNOTATION_RESULTS = "../1-Data/3-annotation/output.csv"
df = pd.read_csv(ANNOTATION_RESULTS)
nlp = spacy.load('en_core_web_sm')
alternative_company_names = {"AMD (Advanced Micro Devices)": "AMD",
                    'Royal Dutch Shell PLC': "Shell",
                    "Samsung Electronics Co., Ltd.": "Samsung",
                    "Goodyear Tire & Rubber Co": "Goodyear",
                    "Sumitomo Rubber Industries": "Sumitomo",
                    "Exxon Mobil Corp.": "Exxon",
                    "General Motors Corp.": "GM",
                    "Ford Motor Co.": "Ford",
                    "Toyota Motor Corp.": "Toyota",
                    "Petro China": "PetroChina",
                    'Volkswagen AG': "VW"}

Clean text from html tags

In [2]:
len(df)

348

In [3]:
def clean_text(html):
    soup = BeautifulSoup(html, "html.parser") # create a new bs4 object from the html data loaded
    for script in soup(["script", "style"]): # remove all javascript and stylesheet code
        script.extract()
    texts = soup.findAll(text=True)
#     import ipdb; ipdb.set_trace()
#     print(len(texts))
    text = ". ".join(t.strip() for t in texts)
    text = unicodedata.normalize("NFKD", text)
    return text
df['text'] = df.apply(lambda row: "{} {}".format(row['title'], clean_text(str(row['full_text']))), axis=1)
df.drop(df[df.text.str.len() < 150].index, inplace=True)

In [4]:
len(df)

340

In [5]:
# COMPANY_NAMES_STOP_WORDS = "PLC|Corp"
with open("company-suffix.txt", "r") as fl:
    text = [i for i in fl.read().split('\n') if not i.startswith('//')]
    COMPANY_NAMES_STOP_WORDS = "(" + "$)|(".join(text) + "$)"

In [6]:
def get_company_names(company):
    company_names = [re.sub(COMPANY_NAMES_STOP_WORDS, '', company, flags=re.IGNORECASE).strip().lower()]
    if company in alternative_company_names:
        company_names.append(alternative_company_names[company].lower())
    #Company is often mentioned by part of it's name. e.g. "Royal Dutch Shell" -> "Shell"
#     company_names = set([company] + [i for i in company.split() if len(i)>2])
    return company_names

In [7]:
# pd.options.display.max_colwidth = 10000
# row = df.loc[df['url'] == 'http://www.sustainablebrands.com/solutionproviders/basf']
# row

## Sentiment Baseline:  
Using the same BoW technique as above  
I exclude texts that are not related to the company or climate change

In [8]:
target_names=('Strongly Negative', 'Negative', 'Neutral', 'Positive', 'Strongly Positive')
cleaned_df = df[(df['Answer.CompanyConfidence'] != 0) & (df['Answer.ClimateConfidence'] != 0)]
cleaned_df = cleaned_df.dropna(subset =["Answer.sentiment"])

# cleaned_df.dropna(subset=['Answer.sentiment'], how='all', inplace = True)
simple_dc = {"Strongly Negative": '0', "Negative": '0', "Neutral": '2', "Positive": '1', "Strongly Positive": '1'}
cleaned_df['simple_sentiment'] = cleaned_df.apply(lambda row: simple_dc[row['Answer.sentiment']] ,axis=1)
print("Total: {} texts".format(len(cleaned_df)))
print("Sentiment counts:")
print(cleaned_df['Answer.sentiment'].value_counts(normalize=True))
print("Simple Sentiment counts:")
print(cleaned_df['simple_sentiment'].value_counts(normalize=True))

Total: 215 texts
Sentiment counts:
Positive             0.362791
Negative             0.232558
Strongly Positive    0.181395
Neutral              0.134884
Strongly Negative    0.088372
Name: Answer.sentiment, dtype: float64
Simple Sentiment counts:
1    0.544186
0    0.320930
2    0.134884
Name: simple_sentiment, dtype: float64


# BoW Again  

taken from https://gist.github.com/bbengfort/044682e76def583a12e6c09209c664a1

In [9]:
import os
import time
import string
import pickle

from operator import itemgetter

from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import pos_tag

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report as clsr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split as tts


def timeit(func):
    """
    Simple timing decorator
    """
    def wrapper(*args, **kwargs):
        start  = time.time()
        result = func(*args, **kwargs)
        delta  = time.time() - start
        return result, delta
    return wrapper


def identity(arg):
    """
    Simple identity function works as a passthrough.
    """
    return arg


class NLTKPreprocessor(BaseEstimator, TransformerMixin):
    """
    Transforms input data by using NLTK tokenization, lemmatization, and
    other normalization and filtering techniques.
    """

    def __init__(self, stopwords=None, punct=None, lower=True, strip=True):
        """
        Instantiates the preprocessor, which make load corpora, models, or do
        other time-intenstive NLTK data loading.
        """
        self.lower      = lower
        self.strip      = strip
        self.stopwords  = set(stopwords) if stopwords else set(sw.words('english'))
        self.punct      = set(punct) if punct else set(string.punctuation)
        self.lemmatizer = WordNetLemmatizer()

    def fit(self, X, y=None):
        """
        Fit simply returns self, no other information is needed.
        """
        return self

    def inverse_transform(self, X):
        """
        No inverse transformation
        """
        return X

    def transform(self, X):
        """
        Actually runs the preprocessing on each document.
        """
        return [
            list(self.tokenize(doc)) for doc in X
        ]

    def tokenize(self, document):
        """
        Returns a normalized, lemmatized list of tokens from a document by
        applying segmentation (breaking into sentences), then word/punctuation
        tokenization, and finally part of speech tagging. It uses the part of
        speech tags to look up the lemma in WordNet, and returns the lowercase
        version of all the words, removing stopwords and punctuation.
        """
        # Break the document into sentences
        for sent in sent_tokenize(document):
            # Break the sentence into part of speech tagged tokens
            for token, tag in pos_tag(wordpunct_tokenize(sent)):
                # Apply preprocessing to the token
                token = token.lower() if self.lower else token
                token = token.strip() if self.strip else token
                token = token.strip('_') if self.strip else token
                token = token.strip('*') if self.strip else token

                # If punctuation or stopword, ignore token and continue
                if token in self.stopwords or all(char in self.punct for char in token):
                    continue
                if tag == "NNP":
                    continue
                # Lemmatize the token and yield
                lemma = self.lemmatize(token, tag)
                yield lemma

    def lemmatize(self, token, tag):
        """
        Converts the Penn Treebank tag to a WordNet POS tag, then uses that
        tag to perform much more accurate WordNet lemmatization.
        """
        tag = {
            'N': wn.NOUN,
            'V': wn.VERB,
            'R': wn.ADV,
            'J': wn.ADJ
        }.get(tag[0], wn.NOUN)

        return self.lemmatizer.lemmatize(token, tag)



@timeit
def build_and_evaluate(X, y, classifier=SGDClassifier, outpath=None, verbose=True):
    """
    Builds a classifer for the given list of documents and targets in two
    stages: the first does a train/test split and prints a classifier report,
    the second rebuilds the model on the entire corpus and returns it for
    operationalization.
    X: a list or iterable of raw strings, each representing a document.
    y: a list or iterable of labels, which will be label encoded.
    Can specify the classifier to build with: if a class is specified then
    this will build the model with the Scikit-Learn defaults, if an instance
    is given, then it will be used directly in the build pipeline.
    If outpath is given, this function will write the model as a pickle.
    If verbose, this function will print out information to the command line.
    """

    @timeit
    def build(classifier, X, y=None):
        """
        Inner build function that builds a single model.
        """
        if isinstance(classifier, type):
            classifier = classifier()

        model = Pipeline([
            ('preprocessor', NLTKPreprocessor()),
            ('vectorizer', TfidfVectorizer(tokenizer=identity, preprocessor=None, lowercase=False)),
            ('classifier', classifier),
        ])

        model.fit(X, y)
        return model

    # Label encode the targets
    labels = LabelEncoder()
    y = labels.fit_transform(y)

    # Begin evaluation
    if verbose: print("Building for evaluation")
    X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2)
    model, secs = build(classifier, X_train, y_train)

    if verbose: print("Evaluation model fit in {:0.3f} seconds".format(secs))
    if verbose: print("Classification Report:\n")

    y_pred = model.predict(X_test)
    print(clsr(y_test, y_pred, target_names=labels.classes_))

    if verbose: print("Building complete model and saving ...")
    model, secs = build(classifier, X, y)
    model.labels_ = labels

    if verbose: print("Complete model fit in {:0.3f} seconds".format(secs))

    if outpath:
        with open(outpath, 'wb') as f:
            pickle.dump(model, f)

        print("Model written out to {}".format(outpath))

    return model


def show_most_informative_features(model, text=None, n=20):
    """
    Accepts a Pipeline with a classifer and a TfidfVectorizer and computes
    the n most informative features of the model. If text is given, then will
    compute the most informative features for classifying that text.
    Note that this function will only work on linear models with coefs_
    """
    # Extract the vectorizer and the classifier from the pipeline
    vectorizer = model.named_steps['vectorizer']
    classifier = model.named_steps['classifier']

    # Check to make sure that we can perform this computation
    if not hasattr(classifier, 'coef_'):
        raise TypeError(
            "Cannot compute most informative features on {} model.".format(
                classifier.__class__.__name__
            )
        )

    if text is not None:
        # Compute the coefficients for the text
        tvec = model.transform([text]).toarray()
    else:
        # Otherwise simply use the coefficients
        tvec = classifier.coef_

    # Zip the feature names with the coefs and sort
    coefs = sorted(
        zip(tvec[0], vectorizer.get_feature_names()),
        key=itemgetter(0), reverse=False
    )

    topn  = zip(coefs[:n], coefs[:-(n+1):-1])

    # Create the output string to return
    output = []

    # If text, add the predicted value to the output.
    if text is not None:
        output.append("\"{}\"".format(text))
        output.append("Classified as: {}".format(model.predict([text])))
        output.append("")

    # Create two columns with most negative and most positive features.
    for (cp, fnp), (cn, fnn) in topn:
        output.append(
            "{:0.4f}{: >15}    {:0.4f}{: >15}".format(cp, fnp, cn, fnn)
        )

    return "\n".join(output)


/Users/bmarchenko/projects/prj-nlp/env/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
lemmatizer = WordNetLemmatizer()
stopwords = set(sw.words('english'))
def lemmatize(token, tag):
    """
    Converts the Penn Treebank tag to a WordNet POS tag, then uses that
    tag to perform much more accurate WordNet lemmatization.
    """
    tag = {
        'N': wn.NOUN,
        'V': wn.VERB,
        'R': wn.ADV,
        'J': wn.ADJ
    }.get(tag[0], wn.NOUN)

    return lemmatizer.lemmatize(token, tag)

def tokenize(document, company_names):
    doc_tokens = []

    for sent in sent_tokenize(document):
        company_in_sent = False
        sent_tokens = []
        # Break the sentence into part of speech tagged tokens
        for token, tag in pos_tag(wordpunct_tokenize(sent)):
            # Apply preprocessing to the token
            token = token.lower()
            token = token.strip()
            token = token.strip('_')
            token = token.strip('*')

            # If punctuation or stopword, ignore token and continue
#             import ipdb; ipdb.set_trace()
            if token in stopwords or all(char in set(string.punctuation) for char in token):
                continue
            if tag == "NNP":
                continue
            # Lemmatize the token and yield
            lemma = lemmatize(token, tag)
            if lemma in company_names:
                company_in_sent = True
            sent_tokens.append(lemma)
        if company_in_sent:
            
            doc_tokens.extend(sent_tokens)
#                 break
#     if len(doc_tokens) == 0:
#         print(document)
#         print(company_names)
#         import ipdb; ipdb.set_trace()
#     print(company_names[0], len(doc_tokens))
    return doc_tokens

In [11]:
get_company_names(list(cleaned_df["company"])[1])
# list(cleaned_df["company"])[1]

['royal dutch shell', 'shell']

In [12]:
# get_company_names(list(cleaned_df["company"])[10])
cleaned_df['lemmatized'] = cleaned_df.apply(lambda row: " ".join(tokenize(row['text'], get_company_names(row["company"]))), axis=1)

In [13]:
list(cleaned_df["lemmatized"])[0]
cleaned_df["Answer.sentiment"].unique()

array(['Positive', 'Negative', 'Neutral', 'Strongly Positive',
       'Strongly Negative'], dtype=object)

In [14]:

X = cleaned_df["text"]
y = cleaned_df["Answer.sentiment"]
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42,
                                                    stratify=y)
model, secs = build_and_evaluate(X,y)
print(show_most_informative_features(model))

Building for evaluation


/Users/bmarchenko/projects/prj-nlp/env/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Evaluation model fit in 16.952 seconds
Classification Report:



/Users/bmarchenko/projects/prj-nlp/env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                   precision    recall  f1-score   support

         Negative       0.45      0.90      0.60        10
          Neutral       0.00      0.00      0.00         8
         Positive       0.33      0.55      0.41        11
Strongly Negative       0.00      0.00      0.00         6
Strongly Positive       0.50      0.12      0.20         8

      avg / total       0.28      0.37      0.28        43

Building complete model and saving ...
Complete model fit in 29.649 seconds
-4.2145             21    5.0548          shale
-4.0779       football    4.5045           file
-3.5971            use    4.4127            sea
-3.5618      apartment    3.9375            ice
-3.3879     management    3.6615       refinery
-3.2350       milkweed    3.5665         motion
-3.0131       emission    3.4758             .”
-2.9990           palm    3.4435        science
-2.9845          power    3.4231        lawsuit
-2.8489           well    3.4054        climate
-2.8455        plastic    3.

/Users/bmarchenko/projects/prj-nlp/env/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [15]:
X = cleaned_df["text"]
y = cleaned_df["simple_sentiment"]
# test_size = 0.2
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42,
#                                                     stratify=y)
model, secs = build_and_evaluate(X,y)
print(show_most_informative_features(model))

Building for evaluation


/Users/bmarchenko/projects/prj-nlp/env/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Evaluation model fit in 18.253 seconds
Classification Report:

             precision    recall  f1-score   support

          0       0.56      0.90      0.69        10
          1       0.96      0.81      0.88        31
          2       0.00      0.00      0.00         2

avg / total       0.82      0.79      0.79        43

Building complete model and saving ...
Complete model fit in 22.287 seconds
-4.1960        project    4.8262          shale
-3.5618      apartment    4.4291              “
-3.3744       football    4.3444       fracking
-3.2894           meat    4.2346          tonne
-3.2490              —    3.9310            oil
-3.2350       milkweed    3.7858             .”
-3.1625       hydrogen    3.7112            bee
-3.1284     protection    3.6193         diesel
-3.1016          speak    3.5783           case
-3.0697    application    3.2539            ban
-2.8599           also    3.2091      pollution
-2.8153       reindeer    3.1147        scandal
-2.7251       ref

/Users/bmarchenko/projects/prj-nlp/env/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [16]:
df = pd.read_csv("../1-Data/3-annotation/compiled_output.csv")
df.columns
df['text'] = df.apply(lambda row: "{} {}".format(row['Input.title'], clean_text(str(row['Input.content']))), axis=1)
df.drop(df[df.text.str.len() < 150].index, inplace=True)

In [18]:
companies = df.groupby("Input.company", as_index=False)
rows_list = []
for num, cp in enumerate(list(companies.groups.keys())):
    cp_df = df.loc[df['Input.company'] == cp]
#     cp_df['lemmatized'] = cp_df.apply(lambda row: " ".join(tokenize(row['text'], get_company_names(row["Input.company"]))), axis=1)

#     cp_df['text'] = cp_df.apply(lambda row: "{} {}".format(row['Input.title'], clean_text(str(row['Input.content']))), axis=1)
    
    res = model.predict(cp_df['text'])
    counter = Counter(res)
    
    if counter[1] > 0:
        mark = counter[0]/counter[1]
    else:
        mark = counter[0]
#     import ipdb; ipdb.set_trace()
    rows_list.append({"company": cp, "pos":Counter(res)[1], "neg": Counter(res)[0], "mark":  mark})
    print(f"{cp}: p: {Counter(res)[1]}, n: {Counter(res)[0]}, {round(mark,3)}")
    
final_df =  pd.DataFrame(rows_list) 

ABB: p: 52, n: 2, 0.038
AMD (Advanced Micro Devices): p: 53, n: 4, 0.075
Amazon.com Inc.: p: 43, n: 4, 0.093
Apple Inc: p: 52, n: 9, 0.173
AuFeminin.com: p: 45, n: 18, 0.4
BASF AG: p: 112, n: 20, 0.179
BMW: p: 98, n: 18, 0.184
BP plc: p: 13, n: 23, 1.769
Bayer AG: p: 41, n: 13, 0.317
Bridgestone Corp.: p: 61, n: 5, 0.082
Bunge: p: 51, n: 11, 0.216
Cargill Inc: p: 35, n: 6, 0.171
Chevron Corp.: p: 23, n: 86, 3.739
Continental AG: p: 60, n: 13, 0.217
Daimler AG: p: 100, n: 20, 0.2
Dow Chemical: p: 40, n: 20, 0.5
Duke Energy: p: 41, n: 6, 0.146
EDS Corp.: p: 29, n: 11, 0.379
Exxon Mobil Corp.: p: 9, n: 39, 4.333
Ford Motor Co.: p: 87, n: 29, 0.333
GazProm: p: 45, n: 21, 0.467
General Motors Corp.: p: 115, n: 21, 0.183
Goodyear Tire & Rubber Co: p: 46, n: 7, 0.152
IBM: p: 55, n: 6, 0.109
Ineos: p: 10, n: 40, 4.0
Johnson & Johnson: p: 50, n: 21, 0.42
Michelin: p: 59, n: 2, 0.034
Microsoft Corp.: p: 49, n: 5, 0.102
Motorola Inc.: p: 6, n: 3, 0.5
Nestle SA: p: 34, n: 6, 0.176
PepsiCo Inc.: p:

In [19]:
counter

Counter({0: 18, 1: 32, 2: 2})

In [20]:
final_df.sort_values(['mark'], ascending=[0])

,company,mark,neg,pos
18,Exxon Mobil Corp.,4.333333,39,9
24,Ineos,4.000000,40,10
12,Chevron Corp.,3.739130,86,23
34,RosNeft,2.310345,67,29
7,BP plc,1.769231,23,13
35,Royal Dutch Shell PLC,1.306122,64,49
32,Petrobras,0.693548,43,62
40,Total SA,0.564103,22,39
42,Volkswagen AG,0.562500,18,32
31,Petro China,0.508197,31,61


# print(f"length before: {len(df)}")
# df['title'] = df['Input.title']
# df['company'] = df['Input.company']
# df['text'] = df.apply(lambda row: "{} {}".format(row['Input.title'], clean_text(str(row['Input.content']))), axis=1)

# for num, row in df.iterrows():
#     cc = find_company_confidence(row)
#     if cc == 0:
        

In [ ]:
# df['company_confidence'] = df.apply(lambda row: find_company_confidence(row) ,axis=1)
# df = df[df['company_confidence'][0] != 0]
# print(f"length after: {len(df)}")

In [ ]:
# df['company_confidence'] = df.apply(lambda row: row['company_confidence'][0] ,axis=1)

In [ ]:
# df = df[df['company_confidence'] != 0]
# print(f"length after: {len(df)}")

In [ ]:
# companies = df.groupby("Input.company", as_index=False)
# rows_list = []
# for num, cp in enumerate(list(companies.groups.keys())):
#     cp_df = df.loc[df['Input.company'] == cp]
#     res = model.predict(cp_df['text'])
#     rows_list.append({"company": cp, "pos":Counter(res)[1], "neg": Counter(res)[0], "mark": Counter(res)[0]/Counter(res)[1] })
#     print(f"{cp}: p: {Counter(res)[1]}, n: {Counter(res)[0]}, {round(Counter(res)[0]/Counter(res)[1],3)}")
    
# final_df =  pd.DataFrame(rows_list) 
# final_df.sort_values(['mark'], ascending=[0])